# 変数
## tensorflow変数基礎
TensorflowではTensor及びVariableで変数が定義されている。  
公式のドキュメントからではないが、一番大きな違いはepoch間で使いまわす変数か、単純にデータをやり取りする中間データ的な意味合いの変数か  
また、Variableはデータを保存して、別のプロセスで動作させるときにも復帰させることが可能

## Variable
### Creation
When you create a Variable you pass a Tensor as its initial value to the Variable() constructor. TensorFlow provides a collection of ops that produce tensors often used for initialization from constants or random values.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

v = tf.Variable([3., 4., 5.], name="v")
w = tf.Variable([6., 7., 8.], name="w")
x = tf.Variable(tf.random_normal([3], stddev=0.35))
add_x = x.assign(tf.add(v, w))
print(v)
print(x)

# 値を参照する時はセッション上で走らせることが必要
# また、セッションの最初に変数を初期化するのも自動でやらないのでやってあげることも必要
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(v)) # => [3, 4, 5]
    print(sess.run(add_x)) # => [9, 11, 13]
    print(sess.run(add_x)) # => [9, 11, 13]

Tensor("v/read:0", shape=(3,), dtype=float32)
Tensor("Variable/read:0", shape=(3,), dtype=float32)
[ 3.  4.  5.]
[  9.  11.  13.]
[  9.  11.  13.]


In [2]:
z = tf.Variable([0., 0., 0.], name="z")
add_z = z.assign_add(v) # calculate z = z + v
c = tf.Variable(0)
countup_op = c.count_up_to(20)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(add_z)) # => [3. 4. 5.]
    print(sess.run(add_z)) # => [6. 8. 10.] operation assign keeps previous run value.
    
    for i in range(20):
        sess.run(countup_op)
    print(sess.run(c)) # 20

[ 3.  4.  5.]
[  6.   8.  10.]
20


### Sharing Variables
変数スコープの範囲外で変数を参照したり共有したりする場合に使う  
一度作るとプロセスが終わるまで消えないので、Jupyterのノートブックで利用する場合は2回目以降はKernel -> Restart x でプロセスを再起動して実験すること

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

def create_a():
    with tf.variable_scope("foo"):
        with tf.variable_scope("bar"):
            a = tf.get_variable("a", initializer=[1.]) # create name=foo/bar/a, initial value is [1]
            print(a.name)
            print(a.initialized_value())

create_a()
            
with tf.variable_scope("foo"):
    b = tf.get_variable("b", initializer=[2.])
    with tf.variable_scope("bar", reuse=True):
        a = tf.get_variable("a")
        print(b)
        c = a + b

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(c)) # => 3

foo/bar/a:0
Tensor("foo/bar/Identity:0", shape=(1,), dtype=float32)
Tensor("foo/b/read:0", shape=(1,), dtype=float32)
[ 3.]


In [1]:
# 使い方としては例えば相互との構造を定義する関数とか

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

def linear(inputs, output_dim):
    input_dim = inputs.get_shape()[1].value
    w = tf.get_variable("weight", initializer=tf.random_normal([input_dim, output_dim]))
    b = tf.get_variable("bias", initializer=tf.zeros([output_dim]))
    return tf.matmul(inputs, w) + b

inputs = tf.Variable([1., 2., 3., 4.], name="inputs")
inputs = tf.expand_dims(inputs, 0)

with tf.variable_scope("layer1"):
    l1 = linear(inputs, output_dim = 10)

with tf.variable_scope("layer2"):
    l2 = linear(l1, output_dim = 4)

print(map(lambda e: e.name, tf.global_variables()))
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run([inputs, l1, l2]))
    

[u'inputs:0', u'layer1/weight:0', u'layer1/bias:0', u'layer2/weight:0', u'layer2/bias:0']
[array([[ 1.,  2.,  3.,  4.]], dtype=float32), array([[ -0.56994236,  -7.30462742,  -4.88215542,   6.30314541,
        -11.01966381,  -1.64621854,   6.24833488,  11.6736908 ,
         -1.67106295,   3.19665885]], dtype=float32), array([[-34.84638214,  10.00866413, -11.08512211,   9.81978989]], dtype=float32)]


In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import tensorflow as tf

def my_op(a, b, c, name=None):
  with tf.name_scope(name, "MyOp", [a, b, c]) as scope:
    a = tf.convert_to_tensor(a, name="a")
    b = tf.convert_to_tensor(b, name="b")
    c = tf.convert_to_tensor(c, name="c")
    d = a + b + c
    return d


In [12]:
a = tf.Variable([1], name="a")
b = tf.constant([2])
c = tf.constant([3])

r = my_op(a, b, c, name="test")

print(tf.global_variables())

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(r))

[<tensorflow.python.ops.variables.Variable object at 0x7fb78b028390>, <tensorflow.python.ops.variables.Variable object at 0x7fb78b0774d0>, <tensorflow.python.ops.variables.Variable object at 0x7fb78c1d2210>, <tensorflow.python.ops.variables.Variable object at 0x7fb78f039150>]
[6]
